In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Fork-Join network example 9
val model = Network("model")

In [ ]:
// Create network
val delay = Delay(model, "Delay1")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)

val jobclass1 = ClosedClass(model, "class1", 10, delay, 0)
val jobclass2 = ClosedClass(model, "class2", 10, delay, 0)

In [ ]:
// Service configurations
// Class 1
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(2.0))
delay.setService(jobclass1, Exp(0.5))

// Class 2
queue1.setService(jobclass2, Exp(1.0))
queue2.setService(jobclass2, Exp(2.0))
delay.setService(jobclass2, Exp(0.2))

In [ ]:
// Set routing matrix
val P = model.initRoutingMatrix()

// Class 1 routing
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

// Class 2 routing
P.set(jobclass2, jobclass2, delay, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, delay, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 53000))
solvers.add(MVA(model))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name.replace("Solver", "")}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This overlapping routes fork-join model demonstrates:

1. **Identical Routing Structure**:
   - Both classes follow the same path: Delay → Fork → {Queue1, Queue2} → Join → Delay
   - No class switching or different routing patterns
   - Complete route overlap between job classes

2. **Differentiated Service Rates**:
   - **Queue1**: Rate 1.0 for both classes (identical)
   - **Queue2**: Rate 2.0 for both classes (identical)
   - **Delay**: Rate 0.5 for Class1, Rate 0.2 for Class2 (different)

3. **Performance Differentiation**:
   - **Class1**: Faster at Delay (rate 0.5), shorter cycles
   - **Class2**: Slower at Delay (rate 0.2), longer cycles
   - Identical parallel processing rates

4. **Resource Competition**:
   - Both classes compete for Queue1 and Queue2 resources
   - FCFS scheduling creates potential blocking between classes
   - Class interactions affect overall performance

5. **Population Balance**:
   - 10 jobs each class (equal populations)
   - Different cycle times create different arrival rates to fork
   - Class2 creates less frequent but competing load

6. **Asymmetric Bottlenecks**:
   - **Class1 bottleneck**: Delay station (rate 0.5)
   - **Class2 bottleneck**: Delay station (rate 0.2) - more severe
   - Parallel processing not limiting for either class

The model represents:
- **Multi-priority systems**: Different urgency levels using same resources
- **Mixed workloads**: Various job types sharing infrastructure
- **Manufacturing**: Different products using same assembly lines
- **Computing systems**: Various process types sharing processors

Key insights:
- **Throughput hierarchy**: Class1 has higher throughput due to faster Delay processing
- **Resource utilization**: Queue1 becomes more utilized due to lower rate (1.0 vs 2.0)
- **Interference effects**: Classes compete in parallel sections but have isolated delays

**Performance Pattern**: Class2's slower Delay processing creates longer response times and higher queue lengths, but both classes experience similar parallel processing delays due to shared resources and identical service rates in the fork-join section.